In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-23 23:37:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-05-23 23:37:51 (1.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

In [7]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
|R11LOHEDYJALTN|          5|            0|          0|   N|                Y|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|                Y|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|                Y|
|R2L15IS24CX0LI|          5|            0|          0|   N|                Y|
|R1DJ8976WPWVZU|          3|            0|          0|   N|     

In [10]:
vine_votes_df=vine_df.filter('total_votes>20')

In [11]:
vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [17]:
helpful_votes_df=vine_votes_df.filter('helpful_votes >= 50')

In [18]:
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
| RZOPM62JMW97V|          2|          103|        108|   N|                Y|
|R1D3NR5GREEXXJ|          5|           57|         58|   N|                Y|
|R2WYKBQS8OR08O|          5|          168|        176|   N|                Y|
|R3EGDTFDMCOOG4|          5|           74|         90|   N|                N|
|R1MAOLI5FJHAFM|          4|          249|        261|   Y|                N|
|R359PHUFU7ZTXV|          5|           68|         72|   N|     

In [31]:
review_df=helpful_votes_df.filter(helpful_votes_df["vine"]== 'Y')

In [32]:
review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1MAOLI5FJHAFM|          4|          249|        261|   Y|                N|
| R6V9SHMMG5M8F|          5|          101|        110|   Y|                N|
|R37PVLT6ELL5J4|          4|          181|        209|   Y|                N|
| R2FSFGWZF24V9|          4|           50|         51|   Y|                N|
|R3SRW1E8J56IGV|          5|          262|        281|   Y|                N|
| RZEQYOT2RE0N7|          4|           75|         80|   Y|                N|
|R2WBPX441TH495|          5|          202|        221|   Y|                N|
|R2BYBSYHS66ZN8|          5|          109|        114|   Y|                N|
|R2IXC6U7W4OCQ9|          5|          185|        201|   Y|                N|
|R3JZJOD2512UVY|          4|           85|        101|   Y|     

In [34]:
review_no_df=helpful_votes_df.filter(helpful_votes_df["vine"]== 'N')

In [35]:
review_no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
| RZOPM62JMW97V|          2|          103|        108|   N|                Y|
|R1D3NR5GREEXXJ|          5|           57|         58|   N|                Y|
|R2WYKBQS8OR08O|          5|          168|        176|   N|                Y|
|R3EGDTFDMCOOG4|          5|           74|         90|   N|                N|
|R359PHUFU7ZTXV|          5|           68|         72|   N|                N|
|R3VM9PM5RHZW3C|          5|           93|         98|   N|     

In [38]:
paying_customers_df = review_df.agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "paying_customer_count")

In [39]:
paying_customers_df.show()

+---------------------+
|paying_customer_count|
+---------------------+
|                  253|
+---------------------+



In [40]:
non_paying_customers_df = review_no_df.agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "non_paying_customer_count")

In [41]:
non_paying_customers_df.show()

+-------------------------+
|non_paying_customer_count|
+-------------------------+
|                    17242|
+-------------------------+



In [43]:
paying_customers_5stars_df = review_df.filter("star_rating=5").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)", "paying_customer_5_star_reviews")

In [44]:
paying_customers_5stars_df.show()

+------------------------------+
|paying_customer_5_star_reviews|
+------------------------------+
|                           103|
+------------------------------+



In [45]:
nonpaying_customers_5stars_df = review_no_df.filter("star_rating=5").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)", "nonpaying_customer_5_star_reviews")

In [46]:
nonpaying_customers_5stars_df.show()

+---------------------------------+
|nonpaying_customer_5_star_reviews|
+---------------------------------+
|                             8763|
+---------------------------------+

